In [1]:
#import tempfile
import os
import datetime
import ocgis 
from ocgis.util.shp_process import ShpProcess
from ocgis.util.shp_cabinet import ShpCabinetIterator
from cdo import *   # python version
cdo = Cdo()

In [2]:
# define the environment
if 'obelix' in os.uname()[1] or 'asterix' in os.uname()[1]:
    HOME = os.getenv('HOME')
    OUT_PATH = '/home/estimr2/nhempelmann/data/extremoscop/'
    CACHE = '/home/estimr2/nhempelmann/cache/'
    IN_DIR = '/home/estimr2/EUROCORDEX/EOBS_v11/'
else: 
    HOME = os.getenv('HOME')
    OUT_DIR = HOME+'/data/extremoscop/'

In [3]:
# define ocgis environment

ocgis.env.OVERWRITE = True
ocgis.env.USE_CFUNITS = True
SHP_DIR = os.path.join(HOME + '/birdhouse/flyingpigeon/flyingpigeon/processes/shapefiles/FRA/')
ocgis.env.DIR_SHPCABINET = SHP_DIR
#ocgis.env.DIR_OUTPUT = OUT_DIR
sc = ocgis.ShpCabinet()
geoms = 'regions-2016'
sci = ShpCabinetIterator(geoms)

In [4]:
ref_19611900 = [datetime.datetime(1961,1,1),datetime.datetime(1990,12,31)]
ref_19712000 = [datetime.datetime(1971,1,1),datetime.datetime(2000,12,31)]
ref_19762005 = [datetime.datetime(1976,1,1),datetime.datetime(2005,12,31)]
vars = ['pr', 'tas', 'tasmin', 'tasmax']
indices = ['R20mm']

In [22]:
# pr_file =  os.path.join(CACHE, 'pr_EOBS_0.22_rot_v11_19500101-20141231_COEDEXformat.nc')
pr_file =  os.path.join(IN_DIR, 'pr_EOBS_0.22_rot_v11_19500101-20141231.nc')

In [23]:
pr_file

'/home/estimr2/EUROCORDEX/EOBS_v11/pr_EOBS_0.22_rot_v11_19500101-20141231.nc'

In [11]:
for indice in indices: 
    calc = [{'func': 'icclim_%s' % (indice), 'name': indice}]
    calc_group = ['year'] # [[12, 1, 2], 'unique'

In [12]:
# eobs conversion to CORDEX format

#import sys
#sys.path.append(os.getenv('HOME')+'/birdhouse/flyingpigeon/flyingpigeon/')
CACHE

'/home/estimr2/nhempelmann/cache/'

In [ ]:
# define ocgis environment

ocgis.env.OVERWRITE = True
ocgis.env.USE_CFUNITS = True
SHP_DIR = os.path.join(HOME + '/birdhouse/flyingpigeon/flyingpigeon/processes/shapefiles/FRA/')
ocgis.env.DIR_SHPCABINET = SHP_DIR
#ocgis.env.DIR_OUTPUT = OUT_DIR
sc = ocgis.ShpCabinet()
geoms = 'regions-2016'
sci = ShpCabinetIterator(geoms)

In [ ]:
ref_19611900 = [datetime.datetime(1961,1,1),datetime.datetime(1990,12,31)]
ref_19712000 = [datetime.datetime(1971,1,1),datetime.datetime(2000,12,31)]
ref_19762005 = [datetime.datetime(1976,1,1),datetime.datetime(2005,12,31)]
vars = ['pr', 'tas', 'tasmin', 'tasmax']
indices = ['R20mm']

In [44]:
#for nc in pr_file:
dimension_map = {'X': {'variable': 'longitude', 'dimension': 'x', 'pos': 2},
                 'Y': {'variable': 'latitude', 'dimension': 'y', 'pos': 1},
                 'T': {'variable': 'time',  'dimension': 'time', 'pos': 0}} #, 'bounds': 'time_bnds'
nc = 'pr_EOBS_0.22_rot_v11_19500101-20141231.nc'
rd = ocgis.RequestDataset(os.path.join(IN_DIR, nc), 'pr', dimension_map = dimension_map )
#rd.inspect()

In [50]:
#select_ugid = [1]
nc = 'pr_EOBS_0.22_rot_v11_19500101-20141231.nc'
for indice in indices:
    for ugid in [1, 2, 3, 4, 5, 6, 7, 11,  13, 14, 15, 16, 17]: #  8, 9, 10, 12,
        scenario = 'eobs'
        prefix = nc.replace('pr', indice).replace('.nc', '')
            # create foulder structure and define output dir:
            
        if not os.path.exists(os.path.join(OUT_PATH , 'timeseries',  indice ,scenario , str(ugid))):
            os.makedirs(os.path.join(OUT_PATH , 'timeseries', indice ,scenario , str(ugid)))
        OUT_DIR = os.path.join(OUT_PATH , 'timeseries', indice ,scenario , str(ugid))
            
        if not os.path.exists(os.path.join(OUT_PATH , 'polygons',  indice ,scenario, str(ugid))):
            os.makedirs(os.path.join(OUT_PATH , 'polygons', indice ,scenario, str(ugid)))
        ocgis.env.DIR_OUTPUT = os.path.join(OUT_PATH , 'polygons', indice ,scenario, str(ugid))
            
        output = os.path.join(OUT_DIR,prefix+'.nc') # cdo output
            
        if not os.path.exists(output):
            try:
                    
                rd = ocgis.RequestDataset(os.path.join(IN_DIR, nc), 'pr' , dimension_map = dimension_map)

                geom_nc = ocgis.OcgOperations(dataset=rd, geom=geoms, output_format='nc', 
                                                  select_ugid = [ugid], prefix=prefix , 
                                                  add_auxiliary_files=False ,calc=calc, 
                                                  calc_grouping=calc_group).execute() #

                cdo.fldmean (input = str(geom_nc) , output = output )

                print ugid, prefix
            except Exception as e:
                print 'failed for file : %s \n %s ' % (prefix, e )
        else: 
            print '%s \t %s already processed' % (ugid, prefix) 

1 R20mm_EOBS_0.22_rot_v11_19500101-20141231
2 R20mm_EOBS_0.22_rot_v11_19500101-20141231
3 R20mm_EOBS_0.22_rot_v11_19500101-20141231
4 R20mm_EOBS_0.22_rot_v11_19500101-20141231
5 R20mm_EOBS_0.22_rot_v11_19500101-20141231
6 R20mm_EOBS_0.22_rot_v11_19500101-20141231
7 R20mm_EOBS_0.22_rot_v11_19500101-20141231
11 R20mm_EOBS_0.22_rot_v11_19500101-20141231
13 R20mm_EOBS_0.22_rot_v11_19500101-20141231
14 R20mm_EOBS_0.22_rot_v11_19500101-20141231
15 R20mm_EOBS_0.22_rot_v11_19500101-20141231
16 R20mm_EOBS_0.22_rot_v11_19500101-20141231
17 R20mm_EOBS_0.22_rot_v11_19500101-20141231


/home/estimr2/nhempelmann/cache/ocgis_output.nc


In [ ]:
cdo.setreftime('1949-12-01,00:00:00,days', input=geom_file, output='/home/nils/data/EOBS/tx_0.22deg_rot_2014_Cordex.nc')
cdo.setname('tasmax', input='/home/nils/data/EOBS/tx_0.22deg_rot_2014_Cordex.nc' , output='/home/nils/data/EOBS/tasmax_EOBS-22_2014.nc')